In [99]:
import pandas as pd
import numpy as np
import sqlalchemy
import psycopg2
from psycopg2 import sql

In [78]:
# This notebook will be mainly used for cleaning and performing high level analysis of school testing data for Tennessee schools for the 2021-2022, 2020-2021, 2017-2018, and 2012-2013 schoo years.


In [79]:
# Import the xslx/csv for each school year and convert to a dataframe
read_dis_sc22_df = pd.read_excel('Data/2022/district_assessment_file_suppressed_upd32323.xlsx')
read_dis_sc21_df = pd.read_csv('Data/2021/district_assessment_file_suppressed_upd422.csv')
read_dis_sc18_df = pd.read_csv('Data/2018/data_2018_district_base.csv')
read_dis_sc13_df = pd.read_excel('Data/2013/data_2013_district_base.xlsx')


In [80]:
dis_sc13_df = read_dis_sc13_df
dis_sc18_df = read_dis_sc18_df
dis_sc21_df = read_dis_sc21_df
dis_sc22_df = read_dis_sc22_df

In [81]:
# Check data types
dis_sc22_df.info()
dis_sc21_df.info()
dis_sc18_df.info()
dis_sc13_df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167631 entries, 0 to 167630
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   year                       167631 non-null  int64 
 1   system                     167631 non-null  int64 
 2   system_name                167631 non-null  object
 3   test                       167631 non-null  object
 4   subject                    167631 non-null  object
 5   grade                      167631 non-null  object
 6   student_group              167631 non-null  object
 7   participation_rate         167631 non-null  object
 8   enrolled                   167631 non-null  int64 
 9   tested                     167631 non-null  int64 
 10  valid_tests                167631 non-null  int64 
 11  n_below                    167631 non-null  object
 12  n_approaching              167631 non-null  object
 13  n_met_expectations         167631 non-null  

In [82]:
# View the top 3 rows of each
display(dis_sc13_df.head(3))
display(dis_sc18_df.head(3))
display(dis_sc21_df.head(3))
display(dis_sc22_df.head(3))


,year,system,system_name,school,school_name,subject,grade,subgroup,valid_tests,n_below_bsc,n_bsc,n_prof,n_adv,pct_below_bsc,pct_bsc,pct_prof,pct_adv,pct_bsc_and_below,pct_prof_adv
0,2013,10,Anderson County,0,District,Math,All Grades,All Students,2908,475,1036,888,509,16.4,35.6,30.5,17.5,52,48
1,2013,10,Anderson County,0,District,Math,All Grades,White,2810,451,1005,861,493,16.1,35.8,30.6,17.5,51.9,48.1
2,2013,10,Anderson County,0,District,Math,All Grades,Hispanic,16,1,4,5,6,6.2,25,31.3,37.5,31.2,68.8


,year,system,system_name,test,subject,grade,subgroup,valid_tests,n_below,n_approaching,n_on_track,n_mastered,pct_below,pct_approaching,pct_on_track,pct_mastered,pct_on_mastered
0,2018,10,Anderson County,EOC,Algebra I,10,All Students,80.0,**,**,**,**,**,**,**,**,3.8
1,2018,10,Anderson County,EOC,Algebra I,10,American Indian or Alaska Native,1.0,*,*,*,*,*,*,*,*,*
2,2018,10,Anderson County,EOC,Algebra I,10,Black/Hispanic/Native American,1.0,*,*,*,*,*,*,*,*,*


,year,system,system_name,test,subject,grade,subgroup,participation_rate,enrolled,tested,valid_tests,n_below,n_approaching,n_on_track,n_mastered,pct_below,pct_approaching,pct_on_track,pct_mastered,pct_on_mastered
0,2021,10,Anderson County,EOC,Algebra I,10,All Students,94,89,84,82,**,**,**,**,**,**,**,**,1.2
1,2021,10,Anderson County,EOC,Algebra I,10,Black or African American,*,4,4,4,*,*,*,*,*,*,*,*,*
2,2021,10,Anderson County,EOC,Algebra I,10,Black/Hispanic/Native American,*,9,8,7,*,*,*,*,*,*,*,*,*


,year,system,system_name,test,subject,grade,student_group,participation_rate,enrolled,tested,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded
0,2022,10,Anderson County,EOC,Algebra I,10,All Students,99,79,78,77,**,**,**,**,**,**,**,**,1.3
1,2022,10,Anderson County,EOC,Algebra I,10,American Indian or Alaska Native,*,1,1,1,*,*,*,*,*,*,*,*,*
2,2022,10,Anderson County,EOC,Algebra I,10,Black or African American,*,2,2,2,*,*,*,*,*,*,*,*,*


In [83]:
## Cleaning the Data ##
# Delete 'school' and 'school_name' column for 2013
dis_sc13_df = dis_sc13_df.drop(['school', 'school_name'], axis=1)
dis_sc13_df.head(3)

,year,system,system_name,subject,grade,subgroup,valid_tests,n_below_bsc,n_bsc,n_prof,n_adv,pct_below_bsc,pct_bsc,pct_prof,pct_adv,pct_bsc_and_below,pct_prof_adv
0,2013,10,Anderson County,Math,All Grades,All Students,2908,475,1036,888,509,16.4,35.6,30.5,17.5,52,48
1,2013,10,Anderson County,Math,All Grades,White,2810,451,1005,861,493,16.1,35.8,30.6,17.5,51.9,48.1
2,2013,10,Anderson County,Math,All Grades,Hispanic,16,1,4,5,6,6.2,25,31.3,37.5,31.2,68.8


In [84]:
# Change valid_tests in 2018 from float to int
dis_sc18_df = dis_sc18_df.astype({'valid_tests': int})
dis_sc18_df.head(3)

,year,system,system_name,test,subject,grade,subgroup,valid_tests,n_below,n_approaching,n_on_track,n_mastered,pct_below,pct_approaching,pct_on_track,pct_mastered,pct_on_mastered
0,2018,10,Anderson County,EOC,Algebra I,10,All Students,80,**,**,**,**,**,**,**,**,3.8
1,2018,10,Anderson County,EOC,Algebra I,10,American Indian or Alaska Native,1,*,*,*,*,*,*,*,*,*
2,2018,10,Anderson County,EOC,Algebra I,10,Black/Hispanic/Native American,1,*,*,*,*,*,*,*,*,*


In [85]:
# Change 2021 df column names to match 2022
dis_sc21_df = dis_sc21_df.rename(columns={'n_mastered':'n_exceeded_expectations','n_on_track':'n_met_expectations','pct_on_track':'pct_met_expectations','pct_mastered':'pct_exceeded_expectations','pct_on_mastered':'pct_met_exceeded','subgroup':'student_group'})
display(dis_sc21_df.head(2))
display(dis_sc22_df.head(2))

,year,system,system_name,test,subject,grade,student_group,participation_rate,enrolled,tested,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded
0,2021,10,Anderson County,EOC,Algebra I,10,All Students,94,89,84,82,**,**,**,**,**,**,**,**,1.2
1,2021,10,Anderson County,EOC,Algebra I,10,Black or African American,*,4,4,4,*,*,*,*,*,*,*,*,*


,year,system,system_name,test,subject,grade,student_group,participation_rate,enrolled,tested,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded
0,2022,10,Anderson County,EOC,Algebra I,10,All Students,99,79,78,77,**,**,**,**,**,**,**,**,1.3
1,2022,10,Anderson County,EOC,Algebra I,10,American Indian or Alaska Native,*,1,1,1,*,*,*,*,*,*,*,*,*


In [86]:
# Show the number of tests for the student groups
# For 2021
df_2021 = dis_sc21_df[dis_sc21_df['grade'].isin(['09', '10', '11', '12'])].groupby('student_group')['valid_tests'].sum().reset_index()

# For 2022
df_2022 = dis_sc22_df[dis_sc22_df['grade'].isin(['09', '10', '11', '12'])].groupby('student_group')['valid_tests'].sum().reset_index()



display(df_2021)
display(df_2022)


,student_group,valid_tests
0,All Students,259502
1,American Indian or Alaska Native,1161
2,Asian,5356
3,Black or African American,60130
4,Black/Hispanic/Native American,90777
5,Economically Disadvantaged,72135
6,English Learner Transitional 1-4,4446
7,English Learners,8133
8,English Learners with Transitional 1-4,12569
9,Female,126349


,student_group,valid_tests
0,All Students,288033
1,American Indian or Alaska Native,1315
2,Asian,6173
3,Black or African American,67587
4,Black/Hispanic/Native American,106519
5,Economically Disadvantaged,75920
6,English Learner Transitional 1-4,3730
7,English Learners,10735
8,English Learners with Transitional 1-4,14463
9,Female,139779


In [87]:
# Compare 2018 columns to 2022
display(dis_sc22_df.head(2))
display(dis_sc18_df.head(2))


,year,system,system_name,test,subject,grade,student_group,participation_rate,enrolled,tested,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded
0,2022,10,Anderson County,EOC,Algebra I,10,All Students,99,79,78,77,**,**,**,**,**,**,**,**,1.3
1,2022,10,Anderson County,EOC,Algebra I,10,American Indian or Alaska Native,*,1,1,1,*,*,*,*,*,*,*,*,*


,year,system,system_name,test,subject,grade,subgroup,valid_tests,n_below,n_approaching,n_on_track,n_mastered,pct_below,pct_approaching,pct_on_track,pct_mastered,pct_on_mastered
0,2018,10,Anderson County,EOC,Algebra I,10,All Students,80,**,**,**,**,**,**,**,**,3.8
1,2018,10,Anderson County,EOC,Algebra I,10,American Indian or Alaska Native,1,*,*,*,*,*,*,*,*,*


In [88]:
# Match 2018 columns to 2022
dis_sc18_df = dis_sc18_df.rename(columns={'n_on_track':'n_met_expectations','n_mastered':'n_exceeded_expectations','pct_on_track':'pct_met_expectations','pct_mastered':'pct_exceeded_expectations','pct_on_mastered':'pct_met_exceeded','subgroup':'student_group'})

display(dis_sc22_df.head(2))
display(dis_sc18_df.head(2))

,year,system,system_name,test,subject,grade,student_group,participation_rate,enrolled,tested,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded
0,2022,10,Anderson County,EOC,Algebra I,10,All Students,99,79,78,77,**,**,**,**,**,**,**,**,1.3
1,2022,10,Anderson County,EOC,Algebra I,10,American Indian or Alaska Native,*,1,1,1,*,*,*,*,*,*,*,*,*


,year,system,system_name,test,subject,grade,student_group,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded
0,2018,10,Anderson County,EOC,Algebra I,10,All Students,80,**,**,**,**,**,**,**,**,3.8
1,2018,10,Anderson County,EOC,Algebra I,10,American Indian or Alaska Native,1,*,*,*,*,*,*,*,*,*


In [89]:
# Compare 2013 columns to 2022
display(dis_sc22_df.head(2))
display(dis_sc13_df.head(2))

,year,system,system_name,test,subject,grade,student_group,participation_rate,enrolled,tested,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded
0,2022,10,Anderson County,EOC,Algebra I,10,All Students,99,79,78,77,**,**,**,**,**,**,**,**,1.3
1,2022,10,Anderson County,EOC,Algebra I,10,American Indian or Alaska Native,*,1,1,1,*,*,*,*,*,*,*,*,*


,year,system,system_name,subject,grade,subgroup,valid_tests,n_below_bsc,n_bsc,n_prof,n_adv,pct_below_bsc,pct_bsc,pct_prof,pct_adv,pct_bsc_and_below,pct_prof_adv
0,2013,10,Anderson County,Math,All Grades,All Students,2908,475,1036,888,509,16.4,35.6,30.5,17.5,52,48
1,2013,10,Anderson County,Math,All Grades,White,2810,451,1005,861,493,16.1,35.8,30.6,17.5,51.9,48.1


In [90]:
# Match 2013 columns to 2022
dis_sc13_df = dis_sc13_df.rename(columns={'n_below_bsc':'n_below','n_bsc':'n_approaching','n_prof':'n_met_expectations','n_adv':'n_exceeded_expectations','pct_below_bsc':'pct_below','pct_bsc':'pct_approaching','pct_prof':'pct_met_expectations','pct_adv':'pct_exceeded_expectations','pct_prof_adv':'pct_met_exceeded','pct_bsc_and_below':'pct_approaching_and_below','subgroup':'student_group'})

display(dis_sc22_df.head(2))
display(dis_sc13_df.head(2))

,year,system,system_name,test,subject,grade,student_group,participation_rate,enrolled,tested,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded
0,2022,10,Anderson County,EOC,Algebra I,10,All Students,99,79,78,77,**,**,**,**,**,**,**,**,1.3
1,2022,10,Anderson County,EOC,Algebra I,10,American Indian or Alaska Native,*,1,1,1,*,*,*,*,*,*,*,*,*


,year,system,system_name,subject,grade,student_group,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_approaching_and_below,pct_met_exceeded
0,2013,10,Anderson County,Math,All Grades,All Students,2908,475,1036,888,509,16.4,35.6,30.5,17.5,52,48
1,2013,10,Anderson County,Math,All Grades,White,2810,451,1005,861,493,16.1,35.8,30.6,17.5,51.9,48.1


In [91]:
# Check data types
dis_sc22_df.info()
dis_sc21_df.info()
dis_sc18_df.info()
dis_sc13_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167631 entries, 0 to 167630
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   year                       167631 non-null  int64 
 1   system                     167631 non-null  int64 
 2   system_name                167631 non-null  object
 3   test                       167631 non-null  object
 4   subject                    167631 non-null  object
 5   grade                      167631 non-null  object
 6   student_group              167631 non-null  object
 7   participation_rate         167631 non-null  object
 8   enrolled                   167631 non-null  int64 
 9   tested                     167631 non-null  int64 
 10  valid_tests                167631 non-null  int64 
 11  n_below                    167631 non-null  object
 12  n_approaching              167631 non-null  object
 13  n_met_expectations         167631 non-null  

In [97]:
# Define the columns to be converted to integers and floats
int_columns = ['n_below', 'n_approaching', 'n_met_expectations', 'n_exceeded_expectations']
float_columns = ['pct_below', 'pct_approaching', 'pct_met_expectations', 'pct_exceeded_expectations', 'pct_approaching_and_below','pct_met_exceeded']

# If the column exists in the dataframes then convert it to the specified data type
def convert_columns_to_int_float(df, int_columns, float_columns):
    for col in int_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
        
    for col in float_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float)
        
    return df

# Convert the specified columns in each DataFrame
dis_sc22_df = convert_columns_to_int_float(dis_sc22_df, int_columns, float_columns)
dis_sc21_df = convert_columns_to_int_float(dis_sc21_df, int_columns, float_columns)
dis_sc18_df = convert_columns_to_int_float(dis_sc18_df, int_columns, float_columns)
dis_sc13_df = convert_columns_to_int_float(dis_sc13_df, int_columns, float_columns)

# Check data types
dis_sc22_df.info()
dis_sc21_df.info()
dis_sc18_df.info()
dis_sc13_df.info() 



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167631 entries, 0 to 167630
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   year                       167631 non-null  int64  
 1   system                     167631 non-null  int64  
 2   system_name                167631 non-null  object 
 3   test                       167631 non-null  object 
 4   subject                    167631 non-null  object 
 5   grade                      167631 non-null  object 
 6   student_group              167631 non-null  object 
 7   participation_rate         167631 non-null  object 
 8   enrolled                   167631 non-null  int64  
 9   tested                     167631 non-null  int64  
 10  valid_tests                167631 non-null  int64  
 11  n_below                    167631 non-null  int32  
 12  n_approaching              167631 non-null  int32  
 13  n_met_expectations         16

In [98]:
# View the top 3 rows of each
display(dis_sc13_df.head(3))
display(dis_sc18_df.head(3))
display(dis_sc21_df.head(3))
display(dis_sc22_df.head(3))

,year,system,system_name,subject,grade,student_group,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_approaching_and_below,pct_met_exceeded
0,2013,10,Anderson County,Math,All Grades,All Students,2908,475,1036,888,509,16.4,35.6,30.5,17.5,52.0,48.0
1,2013,10,Anderson County,Math,All Grades,White,2810,451,1005,861,493,16.1,35.8,30.6,17.5,51.9,48.1
2,2013,10,Anderson County,Math,All Grades,Hispanic,16,1,4,5,6,6.2,25.0,31.3,37.5,31.2,68.8


,year,system,system_name,test,subject,grade,student_group,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded
0,2018,10,Anderson County,EOC,Algebra I,10,All Students,80,0,0,0,0,0.0,0.0,0.0,0.0,3.8
1,2018,10,Anderson County,EOC,Algebra I,10,American Indian or Alaska Native,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,2018,10,Anderson County,EOC,Algebra I,10,Black/Hispanic/Native American,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0


,year,system,system_name,test,subject,grade,student_group,participation_rate,enrolled,tested,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded
0,2021,10,Anderson County,EOC,Algebra I,10,All Students,94,89,84,82,0,0,0,0,0.0,0.0,0.0,0.0,1.2
1,2021,10,Anderson County,EOC,Algebra I,10,Black or African American,*,4,4,4,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,2021,10,Anderson County,EOC,Algebra I,10,Black/Hispanic/Native American,*,9,8,7,0,0,0,0,0.0,0.0,0.0,0.0,0.0


,year,system,system_name,test,subject,grade,student_group,participation_rate,enrolled,tested,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded
0,2022,10,Anderson County,EOC,Algebra I,10,All Students,99,79,78,77,0,0,0,0,0.0,0.0,0.0,0.0,1.3
1,2022,10,Anderson County,EOC,Algebra I,10,American Indian or Alaska Native,*,1,1,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,2022,10,Anderson County,EOC,Algebra I,10,Black or African American,*,2,2,2,0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [108]:
dis_sc22_df.to_csv('district_scores_2022.csv', index=False)
dis_sc21_df.to_csv('district_scores_2021.csv', index=False)
dis_sc18_df.to_csv('district_scores_2018.csv', index=False)
dis_sc13_df.to_csv('district_scores_2013.csv', index=False)


In [111]:
def import_csv_to_postgres(df, csv_file, table_name, host, port, user, password, dbname):
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        dbname=dbname
    )

    # Infer the data types for the table schema
    data_types = {
        'int32': 'integer',
        'int64': 'integer',
        'float64': 'double precision',
        'object': 'text'
    }

    column_defs = ', '.join([f"{col} {data_types[str(df[col].dtype)]}" for col in df.columns])

    # Create a table with the appropriate data types based on the DataFrame
    with conn.cursor() as cur:
        cur.execute(f"DROP TABLE IF EXISTS {table_name};")
        create_table_query = f"CREATE TABLE {table_name} ({column_defs});"
        cur.execute(create_table_query)

        # Import the CSV data into the newly created table
        with open(csv_file, 'r') as f:
            cur.copy_expert(f"COPY {table_name} FROM STDIN WITH CSV HEADER", f)
            conn.commit()

    conn.close()


In [112]:
import_csv_to_postgres(
    df=dis_sc22_df,
    csv_file='district_scores_2022.csv',
    table_name='district_scores_2022',
    host='localhost',
    port=5433,
    user='postgres',
    password='Metcon202#',
    dbname='Tennessee Schools Standardized Scores' 
)

import_csv_to_postgres(
    df=dis_sc21_df,
    csv_file='district_scores_2021.csv',
    table_name='district_scores_2021',
    host='localhost',
    port=5433,
    user='postgres',
    password='Metcon202#',
    dbname='Tennessee Schools Standardized Scores'
)

import_csv_to_postgres(
    df=dis_sc18_df,
    csv_file='district_scores_2018.csv',
    table_name='district_scores_2018',
    host='localhost',
    port=5433,
    user='postgres',
    password='Metcon202#',
    dbname='Tennessee Schools Standardized Scores'
)

import_csv_to_postgres(
    df=dis_sc13_df,
    csv_file='district_scores_2013.csv',
    table_name='district_scores_2013',
    host='localhost',
    port=5433,
    user='postgres',
    password='Metcon202#',
    dbname='Tennessee Schools Standardized Scores' 
)


In [ ]:
# Asterisks represent negligible values in the scope of the data. Replace '*' and '**' with 0 for all columns
# sc22_df.replace(['*', '**'], 0, inplace=True)
# sc21_df.replace(['*', '**'], 0, inplace=True)
